<a href="https://colab.research.google.com/github/mathewrobins/llm-joc/blob/main/Mistral_with_RAG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Create script so output text wraps

In [ ]:
from IPython.display import HTML, display

def set_css():
  display(HTML('''
  <style>
    pre {
      white-space: pre-wrap;
    }
  </style>
  '''))

get_ipython().events.register('pre_run_cell', set_css)


In [ ]:
import locale

def getpreferredencoding(do_setlocale = True):
  return "UTF-8"

locale.getpreferredencoding = getpreferredencoding
locale.getdefaultlocale()


('en_US', 'UTF-8')

# **Create LLM**

The following code is from https://blog.gopenai.com/bye-bye-llama-2-mistral-7b-is-taking-over-get-started-with-mistral-7b-instruct-1504ff5f373c


## Step 1.  Import Libraries

LLM and LangChain libraries

In [ ]:
# -q quiets the output
!pip install -qU kaleido python-multipart uvicorn fastapi==0.99.1 typing-extensions==4.5 torch==2.1
!pip install -qU accelerate bitsandbytes langchain

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.9/79.9 MB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.7/45.7 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.3/60.3 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.4/58.4 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 270.9/270.9 kB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.0/105.0 MB 9.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 798.0/798.0 kB 50.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 68.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 216.6/216.6 kB 23.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.3/48.3 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━

Vector database libraries

In [ ]:
!pip install -qU sentence-transformers chromadb==0.4.2

Import Libraries

In [ ]:
# Import libraries
from transformers import AutoModelForCausalLM, AutoTokenizer
import transformers
import tensorflow
import torch
from langchain.llms import HuggingFacePipeline
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate

device = 'cuda' if torch.cuda.is_available() else 'cpu'

## Step 2.  Download the Mistral 7B Instruct Model and Tokenizer

In [ ]:
model = AutoModelForCausalLM.from_pretrained("mistralai/Mistral-7B-Instruct-v0.2", load_in_4bit=True, device_map='auto')
tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-Instruct-v0.2")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/596 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.94G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.46k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/72.0 [00:00<?, ?B/s]

Mistral is now loaded
3.  Test Mistral

In [ ]:
# Test prompt 1
vegeterian_recipe_prompt = """### Instruction: Act as a gourmet chef.
I have a friend coming over who is a vegetarian.
I want to impress my friend with a special vegetarian dish.
What do you recommend?

Give me two options, along with the whole recipe for each.

 ### Answer:
 """

encoded_instruction = tokenizer(vegeterian_recipe_prompt,  return_tensors="pt", add_special_tokens=True)

model_inputs = encoded_instruction.to(device)

generated_ids = model.generate(**model_inputs, max_new_tokens=1000, do_sample=True, pad_token_id=tokenizer.eos_token_id)
decoded = tokenizer.batch_decode(generated_ids)
print(decoded[0])

/usr/local/lib/python3.10/dist-packages/bitsandbytes/nn/modules.py:226: UserWarning: Input type into Linear4bit is torch.float16, but bnb_4bit_compute_dtype=torch.float32 (default). This will lead to slow inference or training speed.
  warnings.warn(f'Input type into Linear4bit is torch.float16, but bnb_4bit_compute_dtype=torch.float32 (default). This will lead to slow inference or training speed.')


<s> ### Instruction: Act as a gourmet chef.
I have a friend coming over who is a vegetarian.
I want to impress my friend with a special vegetarian dish.
What do you recommend?

Give me two options, along with the whole recipe for each.

 ### Answer:
  Option 1: **Vegetarian Pad Thai**

  #### Ingredients:
  - 8 oz (230g) rice noodles
  - 2 tbsp vegetable oil
  - 1 cup (250 ml) bean sprouts
  - 2 cups (250g) shredded cabbage
  - 1 red bell pepper, sliced
  - 1/2 cup (125 ml) unsalted peanuts, crushed
  - 2 green onions, thinly sliced
  - 3 cloves garlic, minced
  - 1 cup (250 ml) carrots, thinly sliced
  - 1 cup (250 ml) mung bean sprouts
  - 1 cup (250 ml) bean, drained and rinsed
  - 4 eggs
  - 2 tbsp tamarind paste
  - 2 tbsp fish sauce (use soy sauce as a substitute)
  - 1 tbsp brown sugar
  - 1 tbsp soy sauce
  - 1 tbsp rice vinegar
  - 1 tbsp chili paste (use less or omit if preferred)
  - salt and pepper to taste

  #### Instructions:
  1. Cook rice noodles according to the packa

II.  Integrate LangChain with Mistral

Step 1.  Create Text Generation Pipeline

In [ ]:
text_generation_pipeline = transformers.pipeline(
    model=model,
    tokenizer=tokenizer,
    task="text-generation",
    temperature=0.2,
    repetition_penalty=1.1,
    return_full_text=True,
    max_new_tokens=1000,

)

Step 2.  Create an instance of Minstral

In [ ]:
mistral_llm = HuggingFacePipeline(pipeline=text_generation_pipeline)


Step 3:  Create Prompt Template

In [ ]:
prompt_template = """
### Instruction: Act as a LARGE Language Models Expert. I will ask you a QUESTION and give you an AUDIENCE PERSONA, and you will respond with an answer easily understandable by the AUDIENCE PERSONA.

### AUDIENCE PERSONA:
{audience_persona}

### QUESTION:
{question}

 ### Answer:
 """

Step 4:  Create Prompt and LLM Chain.

In [ ]:
# Create prompt from prompt template
prompt = PromptTemplate(
    input_variables=["audience_persona", "question"],
    template=prompt_template,
)

# Create llm chain
llm_chain = LLMChain(llm=mistral_llm, prompt=prompt)

Step 5:  Run LLM Chain

In [ ]:
# Input query to LLM
input_text = {
    "audience_persona": "A 5 year-old child",
    "question": "Explain how Large Language Models work."
}

# Generate and print LLM response
response = llm_chain(input_text)


print(response['text'])

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:381: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.2` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


 Imagine if you have a big box of magic crayons that can write stories all by themselves! That's what large language models are like. They take in words or sentences, remember things they've learned before, and then create new, exciting stories based on that information. Sometimes they might make mistakes, just like when we color outside the lines, but with more practice, they get better and better at creating fun tales!


In [ ]:
print(response)

{'audience_persona': 'A 5 year-old child', 'question': 'Explain how Large Language Models work.', 'text': " Imagine if you have a big box of magic crayons that can write stories all by themselves! That's what large language models are like. They take in words or sentences, remember things they've learned before, and then create new, exciting stories based on that information. Sometimes they might make mistakes, just like when we color outside the lines, but with more practice, they get better and better at creating fun tales!"}


**III. Build a RAG Pipeline**
https://blog.gopenai.com/rag-pipeline-with-mistral-7b-instruct-model-a-step-by-step-guide-138df378a0c2

Step 1.  Create an LLM chain (see above)

In [ ]:
Step 2 Install and Import Libraries

In [ ]:
# Moved the install to the top


In [ ]:
from langchain.docstore.document import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI
from langchain.embeddings.sentence_transformer import SentenceTransformerEmbeddings

Step 3  Get Experimentation Documents.

In [ ]:
persona_prompt = """
As crazy as it sounds, a simple 5-word prompt prefix helps ChatGPT think like any professional and enables you to find better responses on the first try.

To my surprise, I pinpointed the issue with my car by having ChatGPT act and think like a mechanic last month. This means people can now use ChatGPT to think like you and do what you do for them, especially if your task is not overly sophisticated. You can also use it to make your life easier — I will show you how and when in this article.

Curious about how this magic works?

It is called the Persona prompt pattern. Here’s its basic form:

Act as a {Profession/Celebrity/…/Fictional Character},

Do {Task Name}

Examples:

Act as a personal stylist. I need advice on what to wear to a job interview in the creative industry.
Act as a financial advisor. Given my income and expenses, provide me with a personalized budget plan for the next six months.
Now that we have seen the basic persona prompting form, these are 3 scenarios when you should put persona prompt to work:

1. When you need a customized perspective
Code reviewer
Financial analyst
Historical expert
Data Scientist
Etc.
These aren’t just job titles but roles ChatGPT can act as to provide viable solutions to your problems.

Throw one-size-fits-all responses out the window — use Persona Prompt Pattern to mould ChatGPT response to fit a specific point of view. Imagine ChatGPT teaching you about taxes like a tax expert or giving you business tips like an expert who run a business for 10 years would.

It is not just about getting information anymore but about getting information with character.

Example:

Act as a business expert. I’m planning on launching a startup in the e-commerce sector. What are some challenges and tips I should consider?

2. When you know ‘who’ you need but not ‘what’ you need.
Most of us almost always know ‘who’ we need help from better than the ‘what’ we need.

When was the last time you thought a mechanic would help you with your car issue?

All the time.

When was the last time you knew your car issue was the compressor?

Only sometimes.

With the Persona Pattern, you only need to know ‘who’ can help you do the task.

Example:

Act as an interior designer. I’ve moved into a new apartment, and it’s too bland. Can you give me some design ideas?

3. When you want to interact with ChatGPT in an “Adaptive,” not “rigid” manner
Most people have ‘rigid’ interactions with ChatGPT.

How so?

People plainly ask ChatGPT and get on with their day with basic responses. But there is more for you if you ask better. You can shape your next interaction with ChatGPT “adaptive” — if you provide clear guidance.

Sounds intriguing?

Just look at these levels of interaction with ChatGPT:

Rigid Interaction: You plainly ask a question, and ChatGPT “responds” based on a General Persona

Example:

Can you give me a vegetarian recipe?

Flexible interaction (with limited adaptability): You introduce parameters into your question (e.g., behavior or profession), and ChatGPT “adapts” based on your specific input.

Example:

I have a friend coming over who is a vegetarian. Can you suggest a special dish?

Persona Pattern: You instruct ChatGPT to act and think like a persona and respond with the persona’s perspective in an “adaptive” manner.

Example:

Act as a gourmet chef. I have a friend coming over who is a vegetarian. I want to impress my friend with a special vegetarian dish. What do you recommend?

Rigid < Flexible < Persona Prompt.

Interact with flexibility and adaptability in mind.

Limitations of Persona Prompt
Sometimes, the persona pattern might make interesting assumptions and hallucinations. It is tough to spot the assumptions if we know little about the persona’s expertise.

Be specific with your questions to avoid unnecessary assumptions.

Conclusion
You can get much better responses from ChatGPT by simply asking better. You can access tailored expertise at your fingertips using the Persona Prompt pattern. Whether seeking insights from a seasoned financial analyst or fashion advice tailored to creative job interviews, the Persona Prompt is your ticket to open the next door for you.

Thanks for reading.
"""

audience_persona = """
Stop throwing your words to the wrong ears!

Imagine you have a well-crafted message but throw it into diverse minds — people of different ages, professions, upbringings, etc.

The Brutal Truth?

Your well-thought-out message means NOTHING if it doesn’t resonate with the audience.

For Example, how would you explain AI to

A 5-year-old child?
A painter?
A Doctor?
An Engineer?
A Pirate?
People from the 18th century?
This is where ChatGPT can help you — tailoring your message to resonate with a specific audience.

In my last article about Persona Prompt Pattern, we looked into how to use ChatGPT to act and think like a persona, but now we go to the other side. We will see how ChatGPT tailors its response to a particular audience.

The “Audience Persona Pattern”

The Audience Persona Pattern uses the audience’s knowledge to tailor the ChatGPT’s response — so the same response can be tailored based on the audience.

Interesting?

Here’s the Audience Persona Pattern basic form

[Your Question]. Assume I am {Audience Persona}

Examples:

Explain to me how Digital Currency works. Assume that I am a 5-year-old child.
Explain how the car engine works. Assume I am a painter.
Explain the concept of Artificial Intelligence to a Doctor.
Now let’s see how ChatGPT explains what Digital Currency is to the above personas — and compare the results.

Question: Explain to me how Digital Currency works.

Audience Personas:

A 5-year-old child.

ChatGPT explaining Digital Currency to a 5-year-old child
2. A Painter


ChatGPT explaining Digital Currency to a Painter
3. A Doctor


ChatGPT explaining Digital Currency to a Doctor
Conveying Your Crafted Message to an Audience
Suppose you want to convey a specific message to a particular audience. In that case, you can copy and paste it into ChatGPT and ask it to explain it to a particular persona.

Basic form:

[PASTE YOUR MESSAGE HERE]. Explain this to [AUDIENCE PERSONA]

Example:

Message: Description of how Large Language Models Work


Asking ChatGPT to tailor my message to a 5-year-old child
Response: tailored message to a 5-year-old child.


A message tailored for a 5-year-old child
See how ChatGPT tailors your message and uses the audience’s familiar language and examples.

You can do it, too. You can utilize the Audience Persona Pattern to tailor your message to resonate with your audience.

Remember,

If you speak to all, you reach to none.

Happy Tailoring

Thanks For Reading
"""

Step 4 Chunk Documents

In [ ]:
# Create Document object from text documents
docs = [Document(page_content=post) for post in [persona_prompt, audience_persona]]

# Split documents into chunks

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500, chunk_overlap=10, separators=['\n\n', '\n', '.']
)

document_chunks = text_splitter.split_documents(docs)

Step 5:  Download an Embedding Model

In [ ]:
# Downloading embedding model
embedding_model = SentenceTransformerEmbeddings(model_name='BAAI/bge-large-zh-v1.5')

.gitattributes:   0%|          | 0.00/1.52k [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/191 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/27.7k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.00k [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/439k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/394 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/110k [00:00<?, ?B/s]

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

### Step 6. Initiate a Vector Store Instance

In [ ]:
# Initiate a chromadb instance

chroma_db = Chroma.from_documents(document_chunks, embedding_model)

retriever = chroma_db.as_retriever()

### Step 7.  Create Your Question Answering (QA) Chain

In [ ]:
# Prompt template
qa_template = """<s>[INST] You are a helpful assistant.
Use the following context to Answer the question below briefly:

{context}

{question} [/INST] </s>
"""


# Create a prompt instance
QA_PROMPT = PromptTemplate.from_template(qa_template)

# Custom QA Chain
qa_chain = RetrievalQA.from_chain_type(
    llm = mistral_llm,
    retriever=retriever,
    chain_type_kwargs={"prompt": QA_PROMPT}
)

### Step 8.  Query Mistral 7B Instruct Model

In [ ]:
# Your Question
question = "What is better, a rigid or a persona prompt?"
#Example questions
# question = "What is the basic form of a persona prompt pattern"
# question = "What is the basic form of audience persona prompt"
# question = "What is better, a rigid or a persona prompt?"
# question = "What are the limitations of persona pattern?"
# question = "What is the capitol of Washington state?"  Need to fix prompt so that it says it doesn't know, instead it hallucinated an answer.

# Query Mistral 7B Instruct model
response = qa_chain({"query": question})

# Print your result
print(response['result'])



/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:381: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.2` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


A persona prompt can be more engaging and effective than a rigid prompt because it allows for a more tailored and relatable response. With a persona prompt, I can assume the role and perspective of a specific character or profession, which can make the interaction feel more authentic and personalized. This can lead to a more memorable and enjoyable experience for the user. However, it's important to remember that the persona should be appropriate and relevant to the task at hand, and not make any unwarranted assumptions or hallucinations. In general, a persona prompt adds flexibility and adaptability to the interaction, making it more suitable for a wide range of audiences and situations.
